In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.functions import col

# Set the location of the Delta Lake and Kafka packages
delta_package = "io.delta:delta-spark_2.12:3.0.0"  # Replace with the correct Delta version
kafka_package = "org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.1"  # Replace with the correct Spark version
xml_package = "com.databricks:spark-xml_2.12:0.14.0"

# Initialize Spark Session with Delta Lake and Kafka support
spark = SparkSession.builder \
    .appName("test") \
    .config("spark.jars.packages", f"{delta_package},{xml_package}") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .getOrCreate()

:: loading settings :: url = jar:file:/home/spark/.local/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/spark/.ivy2/cache
The jars for the packages stored in: /home/spark/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
com.databricks#spark-xml_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-6e83b90c-26a8-4ca4-9a0f-84c808adcd97;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.0.0 in central
	found io.delta#delta-storage;3.0.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
	found com.databricks#spark-xml_2.12;0.14.0 in central
	found commons-io#commons-io;2.8.0 in central
	found org.glassfish.jaxb#txw2;2.3.4 in central
	found org.apache.ws.xmlschema#xmlschema-core;2.2.5 in central
:: resolution report :: resolve 154ms :: artifacts dl 8ms
	:: modules in use:
	com.databricks#spark-xml_2.12;0.14.0 from central in [default]
	commons-io#commons-io;2.8.0 from central in [default]
	io.delta#delta-spark_2.12;3.0.0 from central in [default]
	io.delta#delta-storage;3.0.0 from central in [defa

In [2]:
raw_df = spark.read.format('parquet').option("mergeSchema", "true").load('hdfs://spark-test1:9000/raw/transactions')
raw_df_20 = raw_df.orderBy(col("_raw_insert_timestamp").desc()).limit(20)
print(raw_df.count())

177


In [3]:
processed_df = spark.read.format('delta').option("mergeSchema", "true").load('hdfs://spark-test1:9000/processed/transactions')
processed_df_20 = processed_df.orderBy(col("_processed_insert_timestamp").desc()).limit(20)
display(processed_df_20.toPandas())

23/12/12 20:53:12 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


,TransactionId,Amount,CustomerId,TransactionDateTime,TransactionDate,Location,Result,_processed_insert_timestamp,_raw_insert_timestamp
0,6,236.880005,76,2023-12-12 20:47:44,2023-12-12,LONDON,SUCCESS,2023-12-12 20:47:58.079552,2023-12-12 20:47:49.281194
1,48,440.510010,74,2023-12-12 20:47:24,2023-12-12,LONDON,SUCCESS,2023-12-12 20:47:58.079552,2023-12-12 20:47:27.955291
2,100,609.280029,73,2023-12-12 20:47:42,2023-12-12,PARIS,FAILURE,2023-12-12 20:47:58.079552,2023-12-12 20:47:46.239144
3,52,322.730011,99,2023-12-12 20:47:26,2023-12-12,SYDNEY,SUCCESS,2023-12-12 20:47:58.079552,2023-12-12 20:47:30.323494
4,93,166.500000,99,2023-12-12 20:47:42,2023-12-12,NEW YORK,SUCCESS,2023-12-12 20:47:58.079552,2023-12-12 20:47:46.239144
5,89,807.570007,52,2023-12-12 20:47:13,2023-12-12,LONDON,FAILURE,2023-12-12 20:47:58.079552,2023-12-12 20:47:17.910460
6,16,965.869995,11,2023-12-12 20:46:47,2023-12-12,PARIS,SUCCESS,2023-12-12 20:47:58.079552,2023-12-12 20:47:00.183180
7,78,831.909973,40,2023-12-12 20:47:05,2023-12-12,SYDNEY,FAILURE,2023-12-12 20:47:58.079552,2023-12-12 20:47:09.468653
8,23,555.929993,69,2023-12-12 20:47:40,2023-12-12,PARIS,FAILURE,2023-12-12 20:47:58.079552,2023-12-12 20:47:44.681837
9,22,991.409973,1,2023-12-12 20:47:05,2023-12-12,LONDON,FAILURE,2023-12-12 20:47:58.079552,2023-12-12 20:47:09.468653
